In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
N_TREES = 30

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Generate Ratings History Database

In [2]:
path = "C:/Users/gonzo/Documents/Out of the Park Developments/OOTP Baseball 23/saved_games/Shredders.lg/import_export/csv/"
current_year = pd.read_csv(path + 'players_career_batting_stats.csv')['year'].max()

historic_path = "C:/Users/gonzo/Documents/Out of the Park Developments/OOTP Baseball 23/saved_games/Shredders.lg/dump/"
ratings = None

bench_coach_flds = ['teach_hitting', 'teach_pitching', 'teach_c', 'teach_if', 'teach_of', 'teach_running', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'manager_value']
hitting_coach_flds = ['teach_hitting', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'hitting_coach_value',  'hitting_focus']
pitching_coach_flds = ['teach_pitching', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'pitching_coach_value', 'pitching_focus']
added_columns = ['hitting_focus_contact', 'hitting_focus_power', 'hitting_focus_patience', 'hitting_focus_neutral', 'pitching_focus_power','pitching_focus_finesse','pitching_focus_groundball','pitching_focus_neutral', 'personality_personable','personality_easygoing','personality_normal','personality_tempermental','personality_controlling']

for year in range(2020, current_year + 1):
    for month in [3]:            
        try:   
            path2_string = 'dump_' + str(year) + '_' + str(month).rjust(2, '0') + '/csv/'
            plyrs = pd.read_csv(historic_path + path2_string + 'players.csv', usecols = ['player_id', 'team_id', 'age', 'throws', 'bats', 'height', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader'])
            plyrs['throws_left'] = np.where(plyrs['throws'] == 2, 1, 0)
            plyrs['throws_right'] = np.where(plyrs['throws'] == 1, 1, 0)
            plyrs['bats_left'] = np.where(plyrs['bats'] == 2, 1, 0)
            plyrs['bats_right'] = np.where(plyrs['bats'] == 1, 1, 0)
            plyrs['bats_switch'] = np.where(plyrs['bats'] == 3, 1, 0)
            plyrs.drop(['throws', 'bats'], axis = 'columns', inplace = True)
            
            coaches = pd.read_csv(historic_path + path2_string + 'coaches.csv').query('team_id != 0')

            coaches['hitting_focus_contact'] = np.where(coaches['hitting_focus'] == 0, 1, 0)
            coaches['hitting_focus_power'] = np.where(coaches['hitting_focus'] == 1, 1, 0)
            coaches['hitting_focus_patience'] = np.where(coaches['hitting_focus'] == 2, 1, 0)
            coaches['hitting_focus_neutral'] = np.where(coaches['hitting_focus'] == 3, 1, 0)

            coaches['pitching_focus_power'] = np.where(coaches['pitching_focus'] == 0, 1, 0)
            coaches['pitching_focus_finesse'] = np.where(coaches['pitching_focus'] == 1, 1, 0)
            coaches['pitching_focus_groundball'] = np.where(coaches['pitching_focus'] == 2, 1, 0)
            coaches['pitching_focus_neutral'] = np.where(coaches['pitching_focus'] == 3, 1, 0)

            coaches['personality_personable'] = np.where(coaches['personality'] == 0, 1, 0)
            coaches['personality_easygoing'] = np.where(coaches['personality'] == 1, 1, 0)
            coaches['personality_normal'] = np.where(coaches['personality'] == 2, 1, 0)
            coaches['personality_tempermental'] = np.where(coaches['personality'] == 3, 1, 0)
            coaches['personality_controlling'] = np.where(coaches['personality'] == 4, 1, 0)

            bench_coaches = coaches.query('occupation == 2')[['team_id'] + bench_coach_flds + added_columns]
            hitting_coaches = coaches.query('occupation == 5')[['team_id'] + hitting_coach_flds + added_columns]
            pitching_coaches = coaches.query('occupation == 4')[['team_id'] + pitching_coach_flds + added_columns]
            plyrs = plyrs.merge(bench_coaches, how = 'left', on = 'team_id').merge(hitting_coaches, how = 'left', on = 'team_id', suffixes = ['_bench_coach', '_hitting_coach']).merge(pitching_coaches, how = 'left', on = 'team_id', suffixes = ['_bench_coach', '_pitching_coach'])
            
            r = pd.read_csv(historic_path + path2_string + 'players_scouted_ratings.csv', header = 0, names = ['player_id', 'team_id', 'league_id', 'position', 'role', 'scout_id', 'Speed', 'Steal', 'Baserunning', 'Ovr Contact', 'Ovr Gap', 'Ovr Eye', 'Ovr Avoid Ks', 'Ovr Get Hit', 'Ovr Power', 'Ovr BABIP', 'vsR Contact', 'vsR Gap', 'vsR Eye', 'vsR Avoid Ks', 'vsR Get Hit', 'vsR Power', 'vsR BABIP', 'vsL Contact', 'vsL Gap', 'vsL Eye', 'vsL Avoid Ks', 'vsL Get Hit', 'vsL Power', 'vsL BABIP', 'Pot Contact', 'Pot Gap', 'Pot Eye', 'Pot Avoid Ks', 'Pot Get Hit', 'Pot Power', 'Pot BABIP', 'Bunt', 'Bunt for Hit', 'GB Hitter Type', 'FB Hitter Type', 'Ovr Stuff', 'Ovr Control', 'Ovr Movement', 'Ovr Balk', 'Ovr HBP', 'Ovr WP', 'vsR Stuff', 'vsR Control', 'vsR Movement', 'vsR Balk', 'vsR HBP', 'vsR WP', 'vsL Stuff', 'vsL Control', 'vsL Movement', 'vsL Balk', 'vsL HBP', 'vsL WP', 'Pot Stuff', 'Pot Control', 'Pot Movement', 'Pot Balk', 'Pot HBP', 'Pot WP', 'Fastball', 'Slider', 'Curveball', 'Screwball', 'Forkball', 'Changeup', 'Sinker', 'Splitter', 'Knuckleball', 'Cutter', 'Circlechange', 'Knucklecurve', 'Pot Fastball', 'Pot Slider', 'Pot Curveball', 'Pot Screwball', 'Pot Forkball', 'Pot Changeup', 'Pot Sinker', 'Pot Splitter', 'Pot Knuckleball', 'Pot Cutter', 'Pot Circlechange', 'Pot Knucklecurve', 'Velocity', 'Arm Slot', 'Stamina', 'Ground/Fly', 'Hold Runners', 'IF Range', 'IF Arm', 'Turn DP', 'OF Range', 'OF Arm', 'Catcher Arm', 'Catcher Ability', 'IF Error', 'OF Error', 'Fld - P', 'Fld - C', 'Fld - 1B', 'Fld - 2B', 'Fld - 3B', 'Fld - SS', 'Fld - LF', 'Fld - CF', 'Fld - RF', 'Ovr', 'Pot', 'Scouting Accuracy'])
#             osa_scout = r.query('scout_id == -1')
#             my_scout = r.query('scout_id != -1')
#             r = my_scout.merge(osa_scout, how = 'right', on = 'player_id', suffixes = ['', '_osa'])
            r = r.merge(plyrs, how = 'inner', on = 'player_id')
    
            # Change month to the next month (so ratings reflect the current date)
            month += 1
            month %= 13
            month = max(month, 1)
            if month == 1:
                year += 1

            r['date'] = np.datetime64(str(year) + '-' + str(month).rjust(2, '0') + '-01')
            r['month'] = month
            r['year'] = year
#             r.drop(['team_id_osa', 'league_id_osa', 'position_osa', 'role_osa', 'scout_id_osa', 'team_id_y'], axis = 'columns', inplace = True)
            ratings = pd.concat([ratings, r])
            
        except:
            break
            
            
pitches = ['Fastball', 'Slider', 'Curveball', 'Screwball', 'Forkball', 'Changeup', 'Sinker', 'Splitter', 'Knuckleball', 'Cutter', 'Circlechange', 'Knucklecurve']
get_num_pitches = np.vectorize(lambda pitches : np.count_nonzero(pitches, axis = 1), excluded = [0])
ratings['num_pitches'] = get_num_pitches(ratings[pitches].values)

# Weighted Average of Catcher Abilities by [team_id, year]

In [3]:
pt = pd.read_csv(path + 'players_career_fielding_stats.csv').query('position == 2')[['player_id', 'year', 'team_id', 'ip']]
r = ratings.query('position == 2')[['player_id', 'team_id_x', 'year', 'Catcher Ability']]
r.rename(columns={'team_id_x':'team_id'}, inplace = True)

pt = pt.merge(r, how = 'inner', on = ['player_id', 'year', 'team_id']).query('ip > 0')

ca = pd.DataFrame(pt.groupby(['year', 'team_id']).apply(lambda x : np.average(x['Catcher Ability'], weights = x['ip']))).reset_index().rename(columns = {0 : 'wa_CA'})
# wa_ca_osa = pd.DataFrame(pt.groupby(['year', 'team_id']).apply(lambda x : np.average(x['Catcher Ability_osa'], weights = x['ip']))).reset_index()

# ca = wa_ca.merge(wa_ca_osa, how = 'inner', on = ['year', 'team_id']).rename(columns={'0_x':'wa_CA', '0_y':'wa_CA_osa'})

# Batting Projection Model

In [4]:
batting = pd.read_csv(path + 'players_career_batting_stats.csv').query('level_id == 1 and (split_id == 2 or split_id == 3)').groupby(['player_id', 'split_id', 'year'], as_index = False).sum().query('pa >= 75')

batting['s'] = batting['h'] - batting['d'] - batting['t'] - batting['hr']
batting['wOBA'] = (batting['bb'] * 0.69 + batting['hp'] * 0.72 + batting['s'] * 0.89 + batting['d'] * 1.27 + batting['t'] * 1.62 + batting['hr'] * 2.1) / (batting['ab'] + batting['bb'] - batting['ibb'] + batting['sf'] + batting['hp'])

main_batting_fields = ['Contact', 'Gap', 'Power', 'Eye', 'Avoid Ks', 'BABIP']
vsr_batting_fields = ['vsR ' + fld for fld in main_batting_fields]
vsl_batting_fields = ['vsL ' + fld for fld in main_batting_fields]
pot_batting_fields = ['Pot ' + fld for fld in main_batting_fields]
other_batting_fields = ['age', 'GB Hitter Type', 'FB Hitter Type', 'bats_left', 'bats_right', 'bats_switch', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader']
coach_fields_hitting = ['teach_hitting_bench_coach', 'teach_running', 'handle_veterans_bench_coach', 'handle_rookies_bench_coach', 'handle_players_bench_coach', 'favor_speed_to_power_bench_coach', 'favor_avg_to_obp_bench_coach', 'favor_defense_to_offense_bench_coach', 'favor_pitching_to_hitting_bench_coach', 'favor_veterans_to_prospects_bench_coach', 'manager_value', 'hitting_focus_contact_bench_coach', 'hitting_focus_power_bench_coach', 'hitting_focus_patience_bench_coach', 'hitting_focus_neutral_bench_coach', 'pitching_focus_power_bench_coach', 'pitching_focus_finesse_bench_coach', 'pitching_focus_groundball_bench_coach', 'pitching_focus_neutral_bench_coach', 'personality_personable_bench_coach', 'personality_easygoing_bench_coach', 'personality_normal_bench_coach', 'personality_tempermental_bench_coach','personality_controlling_bench_coach', 'teach_hitting_hitting_coach','handle_veterans_hitting_coach', 'handle_rookies_hitting_coach', 'handle_players_hitting_coach', 'favor_speed_to_power_hitting_coach', 'favor_avg_to_obp_hitting_coach', 'favor_defense_to_offense_hitting_coach', 'favor_pitching_to_hitting_hitting_coach', 'favor_veterans_to_prospects_hitting_coach', 'hitting_coach_value', 'hitting_focus', 'hitting_focus_contact_hitting_coach', 'hitting_focus_power_hitting_coach', 'hitting_focus_patience_hitting_coach', 'hitting_focus_neutral_hitting_coach', 'pitching_focus_power_hitting_coach', 'pitching_focus_finesse_hitting_coach', 'pitching_focus_groundball_hitting_coach', 'pitching_focus_neutral_hitting_coach', 'personality_personable_hitting_coach', 'personality_easygoing_hitting_coach', 'personality_normal_hitting_coach', 'personality_tempermental_hitting_coach', 'personality_controlling_hitting_coach']

bat_vsr = batting.query('split_id == 3').merge(ratings, how = 'inner', on = ['player_id', 'year'])[['wOBA'] + vsr_batting_fields + other_batting_fields + coach_fields_hitting].fillna(0).values
bat_vsl = batting.query('split_id == 2').merge(ratings, how = 'inner', on = ['player_id', 'year'])[['wOBA'] + vsl_batting_fields + other_batting_fields + coach_fields_hitting].fillna(0).values

# Create artificial data 

dataset = np.concatenate((bat_vsr, bat_vsl))
X = dataset[:, 1:]
y = dataset[:, 0]

woba_prediction = RandomForestRegressor(n_estimators = N_TREES, random_state = 0)
woba_prediction.fit(X, y)

del batting
del bat_vsr
del bat_vsl
del dataset
del X
del y

# Pitching Projection Model

In [5]:
players = pd.read_csv(path + 'players.csv', usecols = ['player_id', 'throws'])
pitching = pd.read_csv(path + 'players_career_pitching_stats.csv').query('level_id == 1 and (split_id == 2 or split_id == 3)').merge(ca, how = 'inner', on = ['year', 'team_id']).groupby(['player_id', 'split_id', 'year'], as_index = False).sum().query('bf >= 60').merge(players, how = 'inner', on = 'player_id')
pitching['fip'] = ((pitching['hra'] * 13) + (3 * (pitching['bb'] + pitching['hp'])) - (2 * pitching['k'])) / pitching['ip'] + 3.2
pitching['k_rate'] = pitching['k'] / pitching['bf']
pitching['bb_rate'] = pitching['bb'] / pitching['bf']
pitching['hr_rate'] = pitching['hra'] / pitching['bf']

main_pitching_fields = ['Stuff', 'Movement', 'Control']
vsr_pitching_fields = ['vsR ' + fld for fld in main_pitching_fields]
vsl_pitching_fields = ['vsL ' + fld for fld in main_pitching_fields]
pot_pitching_fields = ['Pot ' + fld for fld in main_pitching_fields]
pitches = ['Fastball', 'Slider', 'Curveball', 'Screwball', 'Forkball', 'Changeup', 'Sinker', 'Splitter', 'Knuckleball', 'Cutter', 'Circlechange', 'Knucklecurve']
pot_pitches = ['Pot ' + pitch for pitch in pitches]
other_pitching_fields = ['num_pitches', 'Velocity', 'Arm Slot', 'Stamina', 'Ground/Fly', 'age', 'throws_left', 'throws_right', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader']
catcher_abilities = ['wa_CA']
coaches_fields_pitching = ['teach_pitching_bench_coach', 'handle_veterans_bench_coach', 'handle_rookies_bench_coach', 'handle_players_bench_coach', 'favor_speed_to_power_bench_coach', 'favor_avg_to_obp_bench_coach', 'favor_defense_to_offense_bench_coach', 'favor_pitching_to_hitting_bench_coach', 'favor_veterans_to_prospects_bench_coach', 'manager_value', 'hitting_focus_contact_bench_coach', 'hitting_focus_power_bench_coach', 'hitting_focus_patience_bench_coach', 'hitting_focus_neutral_bench_coach', 'pitching_focus_power_bench_coach', 'pitching_focus_finesse_bench_coach', 'pitching_focus_groundball_bench_coach', 'pitching_focus_neutral_bench_coach', 'personality_personable_bench_coach', 'personality_easygoing_bench_coach', 'personality_normal_bench_coach', 'personality_tempermental_bench_coach', 'personality_controlling_bench_coach', 'teach_pitching_pitching_coach', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'pitching_coach_value', 'pitching_focus', 'hitting_focus_contact', 'hitting_focus_power', 'hitting_focus_patience', 'hitting_focus_neutral', 'pitching_focus_power', 'pitching_focus_finesse', 'pitching_focus_groundball', 'pitching_focus_neutral', 'personality_personable', 'personality_easygoing', 'personality_normal', 'personality_tempermental', 'personality_controlling']

pitching_stats = ['fip', 'k_rate', 'bb_rate', 'hr_rate']
non_pitcher_stats = {'fip': 10, 'k_rate': 0, 'bb_rate': 0.3, 'hr_rate': 0.2}

# Generate prediction models for each of the pitching stats
pitching_models = []
for stat in pitching_stats:
    pit_vsr = pitching.query('split_id == 3').merge(ratings, how = 'inner', on = ['player_id', 'year'])[[stat] + vsr_pitching_fields + pitches + other_pitching_fields + catcher_abilities + coaches_fields_pitching].fillna(0).values
    pit_vsl = pitching.query('split_id == 2').merge(ratings, how = 'inner', on = ['player_id', 'year'])[[stat] + vsl_pitching_fields + pitches + other_pitching_fields + catcher_abilities + coaches_fields_pitching].fillna(0).values
    
    # Gather synthetic data for non-pitchers
    non_pitchers = ratings.query('num_pitches == 1').merge(ca, how = 'inner', left_on = ['year', 'team_id_x'], right_on = ['year', 'team_id'])
    non_pitchers[stat] = np.array([non_pitcher_stats[stat] for _ in range(non_pitchers.shape[0])])
    non_pitchers = non_pitchers[[stat] + vsl_pitching_fields + pitches + other_pitching_fields + catcher_abilities + coaches_fields_pitching].fillna(0).values
    
    dataset = np.concatenate((pit_vsr, pit_vsl, non_pitchers))
    X = dataset[:, 1:]
    y = dataset[:, 0]
    
    rf = RandomForestRegressor(n_estimators = N_TREES, random_state = 0)
    rf.fit(X, y)
    pitching_models.append(rf)

del pitching
del pit_vsr
del pit_vsl
del dataset
del X
del y

# Defensive Projection Model

In [6]:
fld = pd.read_csv(path + 'players_career_fielding_stats.csv')
fld = fld.groupby(['player_id', 'year', 'position'], as_index = False).sum().query('ip >= 50')
fld['zr/ip'] = fld['zr'] / fld['ip']
fld['zr/150g'] = fld['zr/ip'] * 9 * 150

positions = ['pitcher', 'catcher', 'first_base', 'second_base', 'third_base', 'shortstop', 'left_field', 'center_field', 'right_field']
main_fields = ['IF Range', 'IF Arm', 'Turn DP', 'OF Range', 'OF Arm', 'Catcher Arm', 'Catcher Ability', 'IF Error', 'OF Error', 'Fld - P', 'Fld - C', 'Fld - 1B', 'Fld - 2B', 'Fld - 3B', 'Fld - SS', 'Fld - LF', 'Fld - CF', 'Fld - RF']
# osa_main_fields = [fld + '_osa' for fld in main_fields]
other_fields = ['height', 'age', 'throws_left', 'throws_right', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader']
coach_fields_fielding = ['teach_c', 'teach_if', 'teach_of', 'handle_veterans_bench_coach', 'handle_rookies_bench_coach', 'handle_players_bench_coach', 'favor_speed_to_power_bench_coach', 'favor_avg_to_obp_bench_coach', 'favor_defense_to_offense_bench_coach', 'favor_pitching_to_hitting_bench_coach', 'favor_veterans_to_prospects_bench_coach', 'manager_value', 'hitting_focus_contact_bench_coach', 'hitting_focus_power_bench_coach', 'hitting_focus_patience_bench_coach', 'hitting_focus_neutral_bench_coach', 'pitching_focus_power_bench_coach', 'pitching_focus_finesse_bench_coach', 'pitching_focus_groundball_bench_coach', 'pitching_focus_neutral_bench_coach', 'personality_personable_bench_coach', 'personality_easygoing_bench_coach', 'personality_normal_bench_coach', 'personality_tempermental_bench_coach', 'personality_controlling_bench_coach']

# One-Hot Encode Positions
for i, p in enumerate(positions):
    fld[p] = np.where(fld['position'] == i + 1, 1, 0)

# Create synthetic data for fielders out of position (2B - RF Only)
bad_fielders = None
bad_zrs = {
    4: -50,
    5: -35,
    6: -70,
    7: -30,
    8: -40,
    9: -35
}

for i in range(4, 10):
    query = ''
    if i <= 6:
        query = '`IF Range` <= 30 and `IF Arm` <= 30'
    else:
        query = '`OF Range` <= 10'
        
    bad = ratings.query(query)
    
    bad_zr = np.array([bad_zrs[i] for _ in range(bad.shape[0])])
    pos = np.zeros((bad.shape[0], 9))
    pos[:, i - 1] = 1
    
    bad['zr/150g'] = bad_zr
    for i, p in enumerate(positions):
        bad[p] = pos[:, i]
        
    bad_fielders = pd.concat([bad_fielders, bad])
        
bad_fielders = bad_fielders[['zr/150g'] + positions + main_fields + other_fields + coach_fields_fielding].fillna(0)
real_fielders = fld.merge(ratings, how = 'inner', on = ['player_id', 'year'])[['zr/150g'] + positions + main_fields + other_fields + coach_fields_fielding].fillna(0)

dataset = pd.concat([real_fielders, bad_fielders]).values

X = dataset[:, 1:]
y = dataset[:, 0]

zr_prediction = RandomForestRegressor(n_estimators = N_TREES, random_state = 0)
zr_prediction.fit(X, y)

del dataset
del X
del y

In [7]:
projection_models = tuple([woba_prediction, zr_prediction] + pitching_models)

pickle.dump(projection_models, open('projection_models.p', 'wb'))